In [1]:
using Profile
using ProfileView
using QuadGK

Gtk-Message: 13:35:53.027: Failed to load module "appmenu-gtk-module"
Gtk-Message: 13:35:53.084: Failed to load module "canberra-gtk-module"
Gtk-Message: 13:35:53.084: Failed to load module "canberra-gtk-module"


This notebook is dedicated to compare the performance for various implementations of functionsd that are expressed as a sum of basis functions. Elements to compare

1. Basis as Tuple vs. Array
2. Looping using zip vs. element interation.
3. Compare Speciificity vs. Generality (e.g.) functions explicitly defined vs a set of basis functions.

In [2]:
# Two functions that may be used for comparison
F(x) = x^2
G(x) = x - 1


# Number of test elements, and a possible test array
N = Int(1e6 + 1)
X = range(0, 1, length=N)


# A function to take timings of functions
function execute_many_times(test_func, N)
    x = rand()
    for i in 1:N
        test_func(x)
    end
end

execute_many_times (generic function with 1 method)

# Basis as Tuple vs Array 

__Conclusion:__ Tuple / array does not/only marginally impacts execution time.

In [3]:
function build_function(weights, basis)
    function (x)
        out = 0.0
        for (wi, bi) in zip(weights, basis)
            out += wi* bi(x)
        end
        return out
    end
end


# Number of basis functions
nmax = 200

# Weights vector and the basis array/tuple
W = rand(nmax)
b_array = [x -> sin(2π*n*x) for n in 1:nmax]
b_tuple = Tuple([x -> sin(2π*n*x) for n in 1:nmax])

# Create functions from basis, and evaluate single time
f_array = build_function(W, b_array)
f_tuple = build_function(W, b_tuple)

execute_many_times(f_array, 1)
execute_many_times(f_tuple, 1)

@time execute_many_times(f_array, N)
@time execute_many_times(f_tuple, N)

println()
x = rand()
@code_warntype f_array(x)
println("="^60)
println("="^60)
@code_warntype f_tuple(x)

  2.689741 seconds
  2.698225 seconds

Variables
  #self#::var"#1#2"{Array{Float64,1},Array{var"#4#6"{Int64},1}}
  x::Float64
  out::Float64
  @_4::Union{Nothing, Tuple{Tuple{Float64,var"#4#6"{Int64}},Tuple{Int64,Int64}}}
  wi::Float64
  bi::var"#4#6"{Int64}
  @_7::Int64

Body::Float64
1 ─       (out = 0.0)
│   %2  = Core.getfield(#self#, :weights)::Array{Float64,1}
│   %3  = Core.getfield(#self#, :basis)::Array{var"#4#6"{Int64},1}
│   %4  = Main.zip(%2, %3)::Base.Iterators.Zip{Tuple{Array{Float64,1},Array{var"#4#6"{Int64},1}}}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_4::Tuple{Tuple{Float64,var"#4#6"{Int64}},Tuple{Int64,Int64}}::Tuple{Tuple{Float64,var"#4#6"{Int64}},Tuple{Int64,Int64}}
│   %10 = Core.getfield(%9, 1)::Tuple{Float64,var"#4#6"{Int64}}
│   %11 = Base.indexed_iterate(%10, 1)::Core.Compiler.PartialStruct(Tuple{Float64,Int64}, Any[Float64, Core.Compiler.Const(2, false)])
│    

In [4]:
x = rand()
@code_warntype f_array(x)
println("="^60)
println("="^60)
@code_warntype f_tuple(x)

Variables
  #self#::var"#1#2"{Array{Float64,1},Array{var"#4#6"{Int64},1}}
  x::Float64
  out::Float64
  @_4::Union{Nothing, Tuple{Tuple{Float64,var"#4#6"{Int64}},Tuple{Int64,Int64}}}
  wi::Float64
  bi::var"#4#6"{Int64}
  @_7::Int64

Body::Float64
1 ─       (out = 0.0)
│   %2  = Core.getfield(#self#, :weights)::Array{Float64,1}
│   %3  = Core.getfield(#self#, :basis)::Array{var"#4#6"{Int64},1}
│   %4  = Main.zip(%2, %3)::Base.Iterators.Zip{Tuple{Array{Float64,1},Array{var"#4#6"{Int64},1}}}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_4::Tuple{Tuple{Float64,var"#4#6"{Int64}},Tuple{Int64,Int64}}::Tuple{Tuple{Float64,var"#4#6"{Int64}},Tuple{Int64,Int64}}
│   %10 = Core.getfield(%9, 1)::Tuple{Float64,var"#4#6"{Int64}}
│   %11 = Base.indexed_iterate(%10, 1)::Core.Compiler.PartialStruct(Tuple{Float64,Int64}, Any[Float64, Core.Compiler.Const(2, false)])
│         (wi = Core.getfield(%11, 1))
│    

## 2. Looping with Zip vs GetIndex

__Conclusion:__ Looping over a range is 15-20% faster than zipping. This result is independent of array or tuple basis.

### With Array Basis

In [5]:
function build_function_zip(weights, basis)
    function (x)
        out = 0.0
        for (wi, bi) in zip(weights, basis)
            out += wi* bi(x)
        end
        return out
    end
end


function build_function_range(weights, basis)
    n = length(weights)
    function (x)
        out = 0.0
        for i in 1:n
            out += weights[i] * basis[i](x)
        end
        return out
    end
end


# Number of basis functions
nmax = 200

# Weights vector and the basis array
W = rand(nmax)
basis = [x -> sin(2π*n*x) for n in 1:nmax]

# Create functions from basis, and evaluate single time
f_zip = build_function_zip(W, basis)
f_range = build_function_range(W, basis)


execute_many_times(f_zip, 1)
execute_many_times(f_range, 1)

@time execute_many_times(f_zip, N)
@time execute_many_times(f_range, N)

  2.628246 seconds
  2.105652 seconds


In [6]:
println()
x = rand()
@code_warntype f_zip(x)
println("="^60)
println("="^60)
@code_warntype f_range(x)


Variables
  #self#::var"#11#12"{Array{Float64,1},Array{var"#16#18"{Int64},1}}
  x::Float64
  out::Float64
  @_4::Union{Nothing, Tuple{Tuple{Float64,var"#16#18"{Int64}},Tuple{Int64,Int64}}}
  wi::Float64
  bi::var"#16#18"{Int64}
  @_7::Int64

Body::Float64
1 ─       (out = 0.0)
│   %2  = Core.getfield(#self#, :weights)::Array{Float64,1}
│   %3  = Core.getfield(#self#, :basis)::Array{var"#16#18"{Int64},1}
│   %4  = Main.zip(%2, %3)::Base.Iterators.Zip{Tuple{Array{Float64,1},Array{var"#16#18"{Int64},1}}}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_4::Tuple{Tuple{Float64,var"#16#18"{Int64}},Tuple{Int64,Int64}}::Tuple{Tuple{Float64,var"#16#18"{Int64}},Tuple{Int64,Int64}}
│   %10 = Core.getfield(%9, 1)::Tuple{Float64,var"#16#18"{Int64}}
│   %11 = Base.indexed_iterate(%10, 1)::Core.Compiler.PartialStruct(Tuple{Float64,Int64}, Any[Float64, Core.Compiler.Const(2, false)])
│         (wi = Core.getf

### With Tuple Basis

In [7]:
function build_function_zip(weights, basis)
    function (x)
        out = 0.0
        for (wi, bi) in zip(weights, basis)
            out += wi* bi(x)
        end
        return out
    end
end


function build_function_range(weights, basis)
    n = length(weights)
    function (x)
        out = 0.0
        for i in 1:n
            out += weights[i] * basis[i](x)
        end
        return out
    end
end


# Number of basis functions
nmax = 200

# Weights vector and the basis array
W = rand(nmax)
basis = Tuple([x -> sin(2π*n*x) for n in 1:nmax])

# Create functions from basis, and evaluate single time
f_zip = build_function_zip(W, basis)
f_range = build_function_range(W, basis)


execute_many_times(f_zip, 1)
execute_many_times(f_range, 1)

@time execute_many_times(f_zip, N)
@time execute_many_times(f_range, N)

  2.666857 seconds
  2.121630 seconds


In [8]:
println()
x = rand()
@code_warntype f_zip(x)
println("="^60)
println("="^60)
@code_warntype f_range(x)


Variables
  #self#::var"#19#20"{Array{Float64,1},NTuple{200,var"#24#26"{Int64}}}
  x::Float64
  out::Float64
  @_4::Union{Nothing, Tuple{Tuple{Float64,var"#24#26"{Int64}},Tuple{Int64,Int64}}}
  wi::Float64
  bi::var"#24#26"{Int64}
  @_7::Int64

Body::Float64
1 ─       (out = 0.0)
│   %2  = Core.getfield(#self#, :weights)::Array{Float64,1}
│   %3  = Core.getfield(#self#, :basis)::NTuple{200,var"#24#26"{Int64}}
│   %4  = Main.zip(%2, %3)::Base.Iterators.Zip{Tuple{Array{Float64,1},NTuple{200,var"#24#26"{Int64}}}}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_4::Tuple{Tuple{Float64,var"#24#26"{Int64}},Tuple{Int64,Int64}}::Tuple{Tuple{Float64,var"#24#26"{Int64}},Tuple{Int64,Int64}}
│   %10 = Core.getfield(%9, 1)::Tuple{Float64,var"#24#26"{Int64}}
│   %11 = Base.indexed_iterate(%10, 1)::Core.Compiler.PartialStruct(Tuple{Float64,Int64}, Any[Float64, Core.Compiler.Const(2, false)])
│         (wi = 

## 3. Generality vs Specifity

__Conclusion__: Large variation in performance, and it varies which is best. In other words, no difference between generalized and specialized functions.

In [9]:
function build_function_general(weights, basis)
    n = length(weights)
    function (x)
        out = 0.0
        for i in 1:n
            out += weights[i] * basis[i](x)
        end
        return out
    end
end

function build_function_special(weights)
    n = length(weights)
    function (x)
        out = 0.0
        for i in 1:n
            out += weights[i] * sin(2π*i*x)
        end
        return out
    end
end

# Number of basis functions
nmax = 200

# Weights vector and the basis array
W = rand(nmax)
basis = Tuple([x -> sin(2π*n*x) for n in 1:nmax])

# Create functions from basis, and evaluate single time
f_general = build_function_general(W, basis)
f_special = build_function_special(W)


execute_many_times(f_general, 1)
execute_many_times(f_special, 1)

@time execute_many_times(f_general, N)
@time execute_many_times(f_special, N)

  2.124550 seconds
  2.092792 seconds


In [10]:
println()
x = rand()
@code_warntype f_general(x)
println("="^60)
println("="^60)
@code_warntype f_special(x)


Variables
  #self#::var"#27#28"{Array{Float64,1},NTuple{200,var"#32#34"{Int64}},Int64}
  x::Float64
  out::Float64
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Float64
1 ─       (out = 0.0)
│   %2  = Core.getfield(#self#, :n)::Int64
│   %3  = (1:%2)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_4 = Base.iterate(%3))
│   %5  = (@_4 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = out::Float64
│   %12 = Core.getfield(#self#, :weights)::Array{Float64,1}
│   %13 = Base.getindex(%12, i)::Float64
│   %14 = Core.getfield(#self#, :basis)::NTuple{200,var"#32#34"{Int64}}
│   %15 = Base.getindex(%14, i)::var"#32#34"{Int64}
│   %16 = (%15)(x)::Float64
│   %17 = (%13 * %16)::Float64
│         (out = %11 + %17)
│         (@_4 = Base.iterate(%3, %10))
│   %20 = 

## 4. Comparison Specific vs. General Inner Product

__Conclusion__: Choose any of the l2 inner product variants, instead of defining the pointwise product beforehand.

In [11]:
function integrate_many_times(N, integrator, args...)
    for i in 1:N
        integrator(args...)
    end
end
        

function pointwise_product(f, g)
    function (x)
        return f(x) * g(x)
    end
end

function adaptive(f)
    return quadgk(f, 0, 1)[1]
end

function l2_inner_product(f, g)
    return quadgk(x -> f(x) * g(x), 0, 1)[1]
end

function l2_inner_product_spec(f, g)
    spec = pointwise_product(f, g)
    return quadgk(spec, 0, 1)[1]
end

function l2_inner_product_spec2(f, g)
    spec = x -> f(x) * g(x)
    return quadgk(spec, 0, 1)[1]
end

function l2_inner_product_spec2(f, g)
    spec = x -> f(x) * g(x)
    return quadgk(spec, 0, 1)[1]
end

function l2_inner_product_spec3(f, g)
    spec = x -> f(x) * g(x)
    return integrate(spec, adaptive)
end


prod_f_g = pointwise_product(F, G)
prod_f_g(0.1)

integrate_many_times(1, integrate, prod_f_g)
integrate_many_times(1, l2_inner_product, F, G)
integrate_many_times(1, l2_inner_product_spec, F, G)
integrate_many_times(1, l2_inner_product_spec2, F, G)
integrate_many_times(1, l2_inner_product_spec3, F, G)



@time integrate_many_times(100000, integrate, prod_f_g)
@time integrate_many_times(100000, l2_inner_product, F, G)
@time integrate_many_times(100000, l2_inner_product_spec, F, G)
@time integrate_many_times(100000, l2_inner_product_spec2, F, G)
@time integrate_many_times(100000, l2_inner_product_spec3, F, G)

LoadError: UndefVarError: integrate not defined

In [12]:
function integrate(f, integrator)
    return integrator(f)
end

integrate (generic function with 1 method)

In [13]:
@code_warntype(l2_inner_product_spec3)

LoadError: expression is not a function call or symbol

In [14]:
@code_warntype integrate(prod_f_g)
println("="^60)
println("="^60)
@code_warntype l2_inner_product(F, G)

Variables
  #self#::Core.Compiler.Const(l2_inner_product, false)
  f::Core.Compiler.Const(F, false)
  g::Core.Compiler.Const(G, false)
  #37::var"#37#38"{typeof(F),typeof(G)}

Body::Float64
1 ─ %1 = Main.:(var"#37#38")::Core.Compiler.Const(var"#37#38", false)
│   %2 = Core.typeof(f)::Core.Compiler.Const(typeof(F), false)
│   %3 = Core.typeof(g)::Core.Compiler.Const(typeof(G), false)
│   %4 = Core.apply_type(%1, %2, %3)::Core.Compiler.Const(var"#37#38"{typeof(F),typeof(G)}, false)
│        (#37 = %new(%4, f, g))
│   %6 = #37::Core.Compiler.Const(var"#37#38"{typeof(F),typeof(G)}(F, G), false)
│   %7 = Main.quadgk(%6, 0, 1)::Tuple{Float64,Float64}
│   %8 = Base.getindex(%7, 1)::Float64
└──      return %8


### Test Types for a Fourier Sine Expansion

In [15]:
# Number of basis functions
nmax = 200

# Weights vector and the basis array
W = rand(nmax)
basis = Tuple([x -> sin(2π*n*x) for n in 1:nmax])

f = build_function(W, basis)
g = build_function(W, basis)
prod_f_g = pointwise_product(f, g)


execute_many_times(f, 1)
execute_many_times(g, 1)
execute_many_times(prod_f_g, 1)


integrate_many_times(1, integrate, prod_f_g)
integrate_many_times(1, l2_inner_product, F, G)
integrate_many_times(1, l2_inner_product_spec, F, G)


@time integrate_many_times(10, integrate, prod_f_g)
@time integrate_many_times(10, l2_inner_product, f, g)
@time integrate_many_times(10, l2_inner_product_spec, f, g)

LoadError: MethodError: no method matching integrate(::var"#35#36"{var"#1#2"{Array{Float64,1},NTuple{200,var"#46#48"{Int64}}},var"#1#2"{Array{Float64,1},NTuple{200,var"#46#48"{Int64}}}})
Closest candidates are:
  integrate(::Any, !Matched::Any) at In[12]:1

## Compare struct functions 

In [32]:
mutable struct FunctionStruct
    weights
    basis
    call
end

FunctionStruct(weights, basis) = FunctionStruct(weights, basis, build_function(weights, basis))

function (fs::FunctionStruct)(x)
    fs.call(x)::Float64
end

# Weights vector and the basis array
nmax = 3
W = rand(nmax)
basis = Tuple([x -> sin(2π*n*x) for n in 1:nmax])

f = build_function(W, basis)
g = FunctionStruct(W, basis, f)
g2 = FunctionStruct(W, basis)


execute_many_times(f, 1)
execute_many_times(g, 1)
execute_many_times(g.call, 1)
execute_many_times(g2.call, 1)



N = 100000
@time execute_many_times(g, N)
@time execute_many_times(g.call, N)
@time execute_many_times(g2.call, N)
@time execute_many_times(f, N)

  0.006129 seconds (200.00 k allocations: 3.052 MiB)
  0.003963 seconds
  0.004363 seconds
  0.004492 seconds


In [18]:
@code_warntype f(x)

Variables
  #self#::var"#1#2"{Array{Float64,1},Tuple{var"#50#52"{Int64},var"#50#52"{Int64},var"#50#52"{Int64}}}
  x::Float64
  out::Float64
  @_4::Union{Nothing, Tuple{Tuple{Float64,var"#50#52"{Int64}},Tuple{Int64,Int64}}}
  wi::Float64
  bi::var"#50#52"{Int64}
  @_7::Int64

Body::Float64
1 ─       (out = 0.0)
│   %2  = Core.getfield(#self#, :weights)::Array{Float64,1}
│   %3  = Core.getfield(#self#, :basis)::Tuple{var"#50#52"{Int64},var"#50#52"{Int64},var"#50#52"{Int64}}
│   %4  = Main.zip(%2, %3)::Base.Iterators.Zip{Tuple{Array{Float64,1},Tuple{var"#50#52"{Int64},var"#50#52"{Int64},var"#50#52"{Int64}}}}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_4::Tuple{Tuple{Float64,var"#50#52"{Int64}},Tuple{Int64,Int64}}::Tuple{Tuple{Float64,var"#50#52"{Int64}},Tuple{Int64,Int64}}
│   %10 = Core.getfield(%9, 1)::Tuple{Float64,var"#50#52"{Int64}}
│   %11 = Base.indexed_iterate(%10, 1)::Core.Compiler.P

In [19]:
@code_warntype g.call(x)

Variables
  #self#::var"#1#2"{Array{Float64,1},Tuple{var"#50#52"{Int64},var"#50#52"{Int64},var"#50#52"{Int64}}}
  x::Float64
  out::Float64
  @_4::Union{Nothing, Tuple{Tuple{Float64,var"#50#52"{Int64}},Tuple{Int64,Int64}}}
  wi::Float64
  bi::var"#50#52"{Int64}
  @_7::Int64

Body::Float64
1 ─       (out = 0.0)
│   %2  = Core.getfield(#self#, :weights)::Array{Float64,1}
│   %3  = Core.getfield(#self#, :basis)::Tuple{var"#50#52"{Int64},var"#50#52"{Int64},var"#50#52"{Int64}}
│   %4  = Main.zip(%2, %3)::Base.Iterators.Zip{Tuple{Array{Float64,1},Tuple{var"#50#52"{Int64},var"#50#52"{Int64},var"#50#52"{Int64}}}}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_4::Tuple{Tuple{Float64,var"#50#52"{Int64}},Tuple{Int64,Int64}}::Tuple{Tuple{Float64,var"#50#52"{Int64}},Tuple{Int64,Int64}}
│   %10 = Core.getfield(%9, 1)::Tuple{Float64,var"#50#52"{Int64}}
│   %11 = Base.indexed_iterate(%10, 1)::Core.Compiler.P